In [ ]:
!pip install vncorenlp

In [ ]:
from vncorenlp import VnCoreNLP
import json, re
from tqdm import tqdm 
import nbimporter

In [ ]:
# input_file = r"D:\NCKH\estimate_question_difficulty\data_processed\văn_processed.txt"
# output_file = r"D:\NCKH\estimate_question_difficulty\Embedding\Output_ws\qa_processed_ws.txt"

# def word_segment(input_file, output_file):
#     rdrsegmenter = VnCoreNLP(
#     r"D:\Download\VnCoreNLP-1.1.1\VnCoreNLP-1.1.1\VnCoreNLP-1.1.1.jar",
#     annotators="wseg",
#     max_heap_size='-Xmx2g'
#     )   
#     with open(input_file, "r", encoding="utf-8") as f:
#         lines = f.readlines()
#     with open(output_file, "w", encoding="utf-8") as f:
#         for line in tqdm(lines, desc="Word Segmenting"):
#             line = line.strip()
#             if line:
#                 segmented = rdrsegmenter.tokenize(line)
#                 f.write(" ".join(segmented[0]) + "\n")
#     print("✅ Word segmentation done.")
def word_segment(input_file, output_file):
    rdrsegmenter = VnCoreNLP(
        r"D:\Download\VnCoreNLP-1.1.1\VnCoreNLP-1.1.1\VnCoreNLP-1.1.1.jar",
        annotators="wseg",
        max_heap_size='-Xmx2g'
    )   
    with open(input_file, "r", encoding="utf-8") as f:
        lines = f.readlines()
    with open(output_file, "w", encoding="utf-8") as f:
        for line in tqdm(lines, desc="Word Segmenting"):
            line = line.strip()
            if line:
                segmented = rdrsegmenter.tokenize(line)
                seg_line = " ".join(segmented[0])
                # 🔹 cleanup: xóa khoảng trắng quanh "_"
                seg_line = re.sub(r"\s*_\s*", "_", seg_line)
                f.write(seg_line + "\n")
    print("✅ Word segmentation done.")

In [ ]:
import json
import re

import json, re

def ws_question(input_file, output_file_ws):
    with open(input_file, "r", encoding="utf-8") as f:
        text = f.read()

    blocks = re.split(r"(?=Câu\s*\d+\s*:)", text)  # tách theo "Câu số:"
    questions = []

    for i, block in enumerate(blocks, 1):
        block = block.strip()
        if not block: 
            continue

        # id
        id_match = re.search(r"Câu\s*(\d+)\s*:", block)
        qid = int(id_match.group(1)) if id_match else i

        # subject
        subject = re.search(r"Môn\s*:\s*(.+)", block)
        subject = subject.group(1).strip() if subject else None

        # question
        question = re.search(r"Câu hỏi\s*:\s*(.+?)(?=\n\s*[A-D]\.|Đáp|Giải thích|$)", block, re.S)
        question = question.group(1).strip().replace("\n", " ") if question else None

        options = re.findall(r"([A-D])\.\s*(.+)", block)
        # loại bỏ trùng lặp giữ nguyên thứ tự
        seen, options_list = set(), []
        for k, v in options:
            opt = f"{k}. {v.strip()}"
            if opt not in seen:
                options_list.append(opt)
                seen.add(opt)

        ans = re.search(r"Đáp[_ ]?án đúng\s*:\s*([A-D])\.\s*(.+)", block)
        answer = f"{ans.group(1)}. {ans.group(2).strip()}" if ans else None


        if subject and question and options_list and answer:
            questions.append({
                "id": qid,
                "subject": subject,
                "question": question,
                "options": options_list,
                "answer": answer
            })

    with open(output_file_ws, "w", encoding="utf-8") as f:
        json.dump(questions, f, ensure_ascii=False, indent=4)

    print(f"✅ Đã chuyển {len(questions)} câu hỏi sang JSON")


In [ ]:
import json
import re

def ori_question(input_file, output_file_ori):
    # Đọc file txt
    with open(input_file, "r", encoding="utf-8") as f:
        text = f.read()

    # Tách các block câu hỏi bằng chuỗi gạch ngang dài
    blocks = [b.strip() for b in text.split("--------------------------------------------------") if b.strip()]
    questions = []
    for i, block in enumerate(blocks, start=1):
        block = block.strip()
        if not block: 
            continue
        # id
        id_match = re.search(r"Câu\s*(\d+)\s*:", block)
        qid = int(id_match.group(1)) if id_match else i
        
        subject_match = re.search(r"Môn\s*:\s*(.+)", block)
        subject = subject_match.group(1).strip() if subject_match else None

        question_match = re.search(r"Câu hỏi\s*:\s*(.+)", block)
        question = question_match.group(1).strip() if question_match else None

        options = re.findall(r"([A-D])\.\s*(.+)", block)
        # loại bỏ trùng lặp giữ nguyên thứ tự
        seen, options_list = set(), []
        for k, v in options:
            opt = f"{k}. {v.strip()}"
            if opt not in seen:
                options_list.append(opt)
                seen.add(opt)

        ans = re.search(r"Đáp[_ ]?án đúng\s*:\s*([A-D])\.\s*(.+)", block)
        answer = f"{ans.group(1)}. {ans.group(2).strip()}" if ans else None

        if subject and question and options_list and answer:
            questions.append({
                "id": qid,
                "subject": subject,
                "question": question,
                "options": options_list,
                "answer": answer,
            })

    with open(output_file_ori, "w", encoding="utf-8") as f:
        json.dump(questions, f, ensure_ascii=False, indent=4)

    print(f"✅ Đã chuyển thành công {len(questions)} câu hỏi sang file json")